# <div align="center"><b> YOLOv11 (You Only Look Once)</b></div>

<div align="right">

<!-- [![Binder](http://mybinder.org/badge.svg)](https://mybinder.org/) -->
[![nbviewer](https://img.shields.io/badge/render-nbviewer-orange?logo=Jupyter)](https://nbviewer.org/github/brunomaso1/vision-transformer/blob/main/notebooks/3.04-yolo.ipynb)
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://githubtocolab.com/brunomaso1/vision-transformer/blob/main/notebooks/3.04-yolo.ipynb)

</div>

* * *

<style>
/* Limitar la altura de las celdas de salida en html */
.jp-OutputArea.jp-Cell-outputArea {
    max-height: 500px;
}
</style>

🛻 <em><font color='MediumSeaGreen'>  Instalaciones: </font></em> 🛻


Este notebook utiliza [Poetry](https://python-poetry.org/) para la gestión de dependencias.
Primero instala Poetry siguiendo las instrucciones de su [documentación oficial](https://python-poetry.org/docs/#installation).
Luego ejecuta el siguiente comando para instalar las dependencias necesarias y activar el entorno virtual:

- Bash:

```bash
poetry install
eval $(poetry env activate)
```

- PowerShell:

```powershell
poetry install
Invoke-Expression (poetry env activate)
```

> 📝 <em><font color='Gray'>Nota:</font></em> Para agregar `pytorch` utilizando Poetry, se utiliza el siguiente comando:
> ```bash
> # Más info: https://github.com/python-poetry/poetry/issues/6409
> poetry source add --priority explicit pytorch_gpu https://download.pytorch.org/whl/cu128 # Seleccionar la wheel adecuada para tu GPU
> poetry add --source pytorch_gpu torch torchvision 
> ```

✋ <em><font color='DodgerBlue'>Importaciones:</font></em> ✋

In [1]:
# Recarga automática de módulos en Jupyter Notebook
%reload_ext autoreload
%autoreload 2

# Recarga automática de módulos en Jupyter Notebook
import os, requests
from pathlib import Path
import random
import time

from PIL import Image
from loguru import logger

# NumPy y utilidades
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

# PyTorch
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
import torchvision.transforms.functional as TF
from ultralytics import YOLO


# Módulos propios
from vision_transformer.plots import plot_confusion_matrix, plot_metric
from vision_transformer.utils import MulticlassAccuracy
from vision_transformer.config import (
    DATA_DIR,
    RANDOM_SEED,
    MODELS_DIR,
    FIGURES_DIR,
    MODEL_DIR_YOLOV11_M,
    METRICS_FILENAME,
    HISTORY_FILENAME,
    PREDICTIONS_FILENAME,
    MLFLOW_URL,
    DATASET_NAME,
    DATASET_VERSION,
    MODEL_NAME_YOLOV11_M,
    PREFECT_URL
)

import mlflow

2025-06-18 18:35:06.718 | INFO     | vision_transformer.config:<module>:15 - PROJ_ROOT path is: E:\Documentos\Git Repositories\vision-transformer
2025-06-18 18:35:06.719 | INFO     | vision_transformer.config:<module>:19 - Actual environment is: dev


🔧 <em><font color='tomato'>Configuraciones:</font></em> 🔧


In [2]:
random.seed(RANDOM_SEED)  # Establece la semilla para la reproducibilidad.
TESTING_MODE = False  # Modo de prueba, si es True, se usa un subconjunto pequeño del dataset.

BATCH_SIZE = 64

# Checkpoints a utilizar
MODEL_NAME = MODEL_NAME_YOLOV11_M
MODEL_FOLDER = MODEL_DIR_YOLOV11_M
CHECKPOINT = MODEL_NAME_YOLOV11_M + ".pt"

# Optimizaciones
# # torch.set_float32_matmul_precision('highest') # Optimización: Establece la precisión de las multiplicaciones de matrices de punto flotante de 32 bits en 'más alta'.
# torch.set_float32_matmul_precision('high') # Optimización: Establece la precisión de las multiplicaciones de matrices de punto flotante de 32 bits en 'alta'.
# # torch.set_float32_matmul_precision('medium') # Optimización: Establece la precisión de las multiplicaciones de matrices de punto flotante de 32 bits en 'media'.
# # torch.backends.cudnn.benchmark = True # Optimización: Para redes CNN (pero como se usa una capa convolucional, se establece en True).

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"  # Establece el dispositivo.
logger.info(f"Dispositivo actual: {DEVICE}")

# MLflow: Configuración de la URI de seguimiento
try:
    response = requests.get(MLFLOW_URL)
    response.raise_for_status()  # Verifica si la solicitud fue exitosa.
    logger.success("Conexión a MLflow establecida correctamente.")
    os.environ["MLFLOW_TRACKING_URI"] = MLFLOW_URL  # Configura la URI de seguimiento de MLflow.
    os.environ["MLFLOW_EXPERIMENT_NAME"] = CHECKPOINT.replace("/", "_")  # Configura el nombre del experimento de MLflow.
    os.environ["MLFLOW_TAGS"] = '{"model_family": "swinv2"}'
except Exception as e:
    logger.error(f"Error al conectar con MLflow. Tienes levantado el servidor de MLflow?")
    raise SystemExit(f"Error al conectar con MLflow: {e}")

# Prefect: Configuración de Prefect
try:
    response = requests.get(PREFECT_URL)
    response.raise_for_status()  # Verifica si la solicitud fue exitosa.
    logger.success("Conexión a Prefect establecida correctamente.")
except Exception as e:
    logger.error(f"Error al conectar con Prefect. Tienes levantado el servidor de Prefect?")
    raise SystemExit(f"Error al conectar con Prefect: {e}")

2025-06-18 18:35:12.306 | INFO     | __main__:<module>:18 - Dispositivo actual: cuda
2025-06-18 18:35:14.314 | SUCCESS  | __main__:<module>:24 - Conexión a MLflow establecida correctamente.
2025-06-18 18:35:16.321 | SUCCESS  | __main__:<module>:36 - Conexión a Prefect establecida correctamente.


<div align="center">✨Datos del proyecto:✨</div>

<p></p>

<div align="center">

| Subtitulo       | *Fine-tuning* del modelo swimv2 sobre el dataset EuroSAT                                                                       |
| --------------- | -------------------------------------------------------------------------------------------------------------------------------------- |
| **Descrpción**  | <small>Análisis exploratorio del proceso de *fine-tuning* del swimv2 sobre el EuroSAT<br/>- *Tarea:* `Clasificación`<br/>- *Modelo*: `yolov11-cls`<br/> - *Dataset*: `EuroSAT` </small>|
<!-- | **Autor** | <small>[Nombre] ([correo]) </small>                                                                                                 | -->

</div>

## Tabla de contenidos
0. [Pasos previos](#pasos-previos)
1. [Carga de datos](#carga-de-datos)
2. [Carga del modelo](#carga-del-modelo)
3. [Preprocesamiento del dataset](#preprocesamiento)
4. [Definicion de las metricas de evaluacion](#metricas)
5. [Entrenamiento del modelo](#entrenamiento)
6. [Resultados](#resultados)

## 0. Pasos previos <a id="pasos-previos"></a>

Ejecuta desde la raíz del proyecto para descargar el dataset EuroSAT:

```bash
python -m vision_transformer.flows.test_flow
```

## 1. Carga de datos <a name="carga-de-datos"></a>

Para el entrenamiento de YOLO no fue necesario la carga del dataset, ya que la utilizando la API de Ultralytics para el entrenamiento del modelo solo necesitamos pasar el directorio del mismo.

## 2. Carga del modelo <a name="carga-del-modelo"></a>

In [6]:
model = YOLO(MODEL_FOLDER / CHECKPOINT)
print(model)

YOLO(
  (model): ClassificationModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C3k2(
        (cv1): Conv(
          (conv): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(192, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track

## 3. Preprocesamiento del dataset <a name="preprocesamiento-del-dataset"></a>

Ultralytics provee automaticamente una serie de aumentaciones que son aplicadas de manera online. En el presente notebook decidimos conservarlas ya que YOLO se utiliza como modelo baseline de comparacion para los modelos basados en Vision Transformers.

## 4. Definición de las métricas de evaluación <a name="metrics"></a>

Para la evaluacion del modelo utilizamos las mismas metricas que aquellas utilizadas para evaluar los modelos basados en Vision Transformers entrenados en los restantes notebooks de el presente proyecto. En este caso, sin embargo, no utilizamos la libreria evaluate de Huggingface, sino que calculamos las metricas de evaluacion utilizando SciKit-Learn sobre las predicciones hechas por el modelo final.

## 5. Entrenamiento del modelo <a name="entrenamiento"></a>

In [7]:
results = model.train(
    data=os.path.join(DATA_DIR, "processed/EuroSAT_RGB_huggingface"),
    epochs=50,
    val=True,
    patience=20,
    device=0,
    project=MODEL_FOLDER,
    optimizer="auto",
    degrees=5,
)

Ultralytics 8.3.156  Python-3.13.3 torch-2.7.1+cu128 CUDA:0 (NVIDIA GeForce RTX 4080 SUPER, 16376MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=E:\Documentos\Git Repositories\vision-transformer\data\processed/EuroSAT_RGB_huggingface, degrees=5, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=E:\Documentos\Git Repositories\vision-transformer\models\yolo11m-cls\yolo11m-cls.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train,

100%|██████████| 5.35M/5.35M [00:00<00:00, 14.1MB/s]


AMP: checks passed 
train: Fast image access  (ping: 0.00.0 ms, read: 42.821.0 MB/s, size: 3.1 KB)


train: Scanning E:\Documentos\Git Repositories\vision-transformer\data\processed\EuroSAT_RGB_huggingface\train... 24300 images, 0 corrupt: 100%|██████████| 24300/24300 [00:03<00:00, 6772.16it/s]


train: New cache created: E:\Documentos\Git Repositories\vision-transformer\data\processed\EuroSAT_RGB_huggingface\train.cache
val: Fast image access  (ping: 0.00.0 ms, read: 58.518.8 MB/s, size: 3.1 KB)


...

      50/50      2.79G    0.03937         12        224: 100%|██████████| 1519/1519 [00:49<00:00, 30.42it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 85/85 [00:01<00:00, 75.86it/s]

                   all      0.984          1



50 epochs completed in 0.768 hours.
Optimizer stripped from E:\Documentos\Git Repositories\vision-transformer\models\yolo11m-cls\train\weights\last.pt, 20.9MB
Optimizer stripped from E:\Documentos\Git Repositories\vision-transformer\models\yolo11m-cls\train\weights\best.pt, 20.9MB

Validating E:\Documentos\Git Repositories\vision-transformer\models\yolo11m-cls\train\weights\best.pt...
Ultralytics 8.3.156  Python-3.13.3 torch-2.7.1+cu128 CUDA:0 (NVIDIA GeForce RTX 4080 SUPER, 16376MiB)
YOLO11m-cls summary (fused): 57 layers, 10,354,442 parameters, 0 gradients, 39.3 GFLOPs
WARNING Dataset 'split=val' not found, using 'split=test' instead.
train: E:\Documentos\Git Repositories\vision-transformer\data\processed\EuroSAT_RGB_huggingface\train... found 24300 images in 10 classes  
val: E:\Documentos\Git Repositories\vision-transformer\data\processed\EuroSAT_RGB_huggingface\test... found 2700 images in 10 classes  
test: E:\Documentos\Git Repositories\vision-transformer\data\processed\EuroSAT

               classes   top1_acc   top5_acc: 100%|██████████| 85/85 [00:01<00:00, 61.83it/s]


                   all      0.985          1
Speed: 0.0ms preprocess, 0.3ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to E:\Documentos\Git Repositories\vision-transformer\models\yolo11m-cls\train
🏃 View run train at: http://localhost:8080/#/experiments/6/runs/a90cccee613d4f55ba2900a252695e59
🧪 View experiment at: http://localhost:8080/#/experiments/6
MLflow: results logged to http://localhost:8080
MLflow: disable with 'yolo settings mlflow=False'


In [ ]:
# Load saved weights
model = YOLO(MODELS_DIR / MODEL_FOLDER / "weights/best.pt")

In [ ]:
logger.info("Evaluando el modelo...")

test_dir = os.path.join(DATA_DIR, "processed/EuroSAT_RGB_huggingface/test")

# Load images and corresponding labels from folders
class_names = sorted(os.listdir(test_dir))  # assumes folder names = class labels
class_to_idx = {name: idx for idx, name in enumerate(class_names)}

images = []
y_true = []

for class_name in class_names:
    class_path = Path(test_dir) / class_name
    for img_file in class_path.glob("*"):
        if img_file.suffix.lower() in [".jpg", ".jpeg", ".png"]:
            images.append(str(img_file))
            y_true.append(class_to_idx[class_name])

# Predict label using the trained model
start_time = time.time()
y_pred = []

for img_path in images:
    result = model.predict(source=img_path, imgsz=224, verbose=False)
    pred_class = result[0].probs.top1
    y_pred.append(pred_class)

runtime = time.time() - start_time

# Compute metrics
eval_accuracy = accuracy_score(y_true, y_pred)
eval_f1 = f1_score(y_true, y_pred, average="weighted")
eval_precision = precision_score(y_true, y_pred, average="weighted")
eval_recall = recall_score(y_true, y_pred, average="weighted")
eval_runtime = runtime
eval_samples_per_second = len(images) / runtime

# Save metrics as a dataframe
metrics = {
    "eval_loss": None,
    "eval_model_preparation_time": None,
    "eval_accuracy": eval_accuracy,
    "eval_f1": eval_f1,
    "eval_precision": eval_precision,
    "eval_recall": eval_recall,
    "eval_runtime": eval_runtime,
    "eval_samples_per_second": eval_samples_per_second,
    "eval_steps_per_second": None,
}

logger.info("Evaluación finalizada. Métricas:")

metrics_df = pd.DataFrame([metrics])
metrics_df.to_csv(MODELS_DIR / MODEL_FOLDER / METRICS_FILENAME, index=False)
metrics_df

2025-06-17 12:17:07.449 | INFO     | __main__:<module>:1 - Evaluando el modelo...
2025-06-17 12:18:03.063 | INFO     | __main__:<module>:51 - Evaluación finalizada. Métricas:


,eval_loss,eval_model_preparation_time,eval_accuracy,eval_f1,eval_precision,eval_recall,eval_runtime,eval_samples_per_second,eval_steps_per_second
0,None,None,0.981481,0.981528,0.981831,0.981481,55.530458,48.621965,None


In [42]:
# Guardamos las predicciones del modelo en el conjunto de test
results_df = pd.DataFrame(
    {
        "y_true": [model.names[i] for i in y_true],
        "y_pred": [model.names[i] for i in y_pred],
    }
)
results_df.to_csv(MODEL_FOLDER / PREDICTIONS_FILENAME, index=False)

## 6. Resultados <a name="resultados"></a>

In [43]:
history = pd.read_csv(MODEL_FOLDER / HISTORY_FILENAME)
results_df = pd.read_csv(MODEL_FOLDER / PREDICTIONS_FILENAME)
y_true = results_df["y_true"].values
y_pred = results_df["y_pred"].values

In [ ]:
plot_confusion_matrix(
    y_true=y_true,
    y_pred=y_pred,
    labels=sorted(set(y_true)),
    filename=CHECKPOINT.replace("/", "-").replace(".pt", "") + "_confusion_matrix",
    dirpath=FIGURES_DIR / MODEL_NAME,
    show_as_percentaje=True
)

In [ ]:
# Filtramos solo las filas que tienen datos útiles
filtered_history = history.copy()
filtered_history = filtered_history[filtered_history["epoch"].notna()]

# Plot de accuracy
if "metrics/accuracy_top1" in filtered_history.columns:
    plot_metric(
        filtered_history[filtered_history["metrics/accuracy_top1"].notna()],
        x_col="epoch",
        y_cols=["metrics/accuracy_top1"],
        y_labels=["Accuracy de evaluación"],
        title="Accuracy por época",
        filename=CHECKPOINT.replace("/", "-").replace(".pt", "") + "_accuracy_plot",
        dirpath=FIGURES_DIR / MODEL_NAME,
    )